
1. V1: Baseline single model (Fully uses HF Framework)
2. V2: Baseline ranker model (HF models integrated to FastAI)
3. V3: Same approach as V1. More reliable training data.

In [ ]:
import pandas as pd, numpy as np
import wt_text_processing_utils as wtp_utils
import torch
torch.set_grad_enabled(False)

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, default_data_collator
data_collator = default_data_collator

mname = "../input/hatebert-regression-single-model/hatebert_reg_baseline/checkpoint-1000"
#They are all in the same range. MSELoss wise, 250 performed better
tokenizer = AutoTokenizer.from_pretrained(mname)

In [ ]:
from datasets import Dataset
key1, key2 = "tclean", None

#Allows truncation. So encoded ds and actual ds should share the same size. 
def preprocess_function(examples):
    if key2 is None:
        return tokenizer(examples[key1], padding=True, truncation=True, max_length=512) 
    return tokenizer(examples[key1], examples[key2], padding=True, truncation=True, max_length=512) 

df = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
df["tclean"] = wtp_utils.preprocess_text(df["text"])

In [ ]:
from scipy.stats import rankdata

model = AutoModelForSequenceClassification.from_pretrained(mname)

args = TrainingArguments("dummy", report_to=["tensorboard"], per_device_eval_batch_size=256)
trainer = Trainer(model, args, data_collator=data_collator, tokenizer=tokenizer)

test_ds = Dataset.from_pandas(df)
test_ds = test_ds.map(preprocess_function, batched=True)
preds = trainer.predict(test_ds)
ranks = preds.predictions[:, 0]

df["score"] = rankdata(ranks, method='ordinal')

In [ ]:
df[["comment_id", "score"]].to_csv("submission.csv", index=False)